In [167]:
no_reviews = 100000     # no of reviews that will be read from file. 
max_review_length = 100 # no of words per review.  reviews will be  truncated or padded to be of this length.
max_words = 5000  
validation_samples = 6000
# this is the size of the index (i.e. most common top words that will be used as features)
training_samples = 10000                        # note code assumes there are enough words in reviews.
embedding_size = 300    # size of embedding.
prop_val = 0.2          # prop of dat for validation set
no_epochs = 8           # N

import pandas as pd
import numpy as np
import os
import random
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import text
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Embedding, Flatten, Dense, LSTM, Dropout, Bidirectional, Conv1D, MaxPooling1D
from keras.utils import to_categorical
import gensim
from gensim.models import KeyedVectors
from gensim.parsing.preprocessing import remove_stopwords
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
embedding_size = 300    # size of embedding.

In [168]:
df = pd.read_csv (r'C:\Users\Avers\Documents\Data\tripadvisor_hotel_reviews.csv', skiprows=0)
df.head()
df = pd.DataFrame(df)
reviews = df['Review']
ratings = df['Rating']

In [169]:
# df["split"] = df.apply(lambda x: "train" if random.randrange(0,100) > 10 else "valid", axis=1)
# df["split"].value_counts()


In [170]:
# reviews = df[df["split"] == "train"]
# ratings = df[df["split"] == "valid"]

In [171]:
text_token = df['Review'].apply(gensim.utils.simple_preprocess)

In [172]:
text_token.shape
print(text_token)

0        [nice, hotel, expensive, parking, got, good, d...
1        [ok, nothing, special, charge, diamond, member...
2        [nice, rooms, not, experience, hotel, monaco, ...
3        [unique, great, stay, wonderful, time, hotel, ...
4        [great, stay, great, stay, went, seahawk, game...
                               ...                        
20486    [best, kept, secret, rd, time, staying, charm,...
20487    [great, location, price, view, hotel, great, q...
20488    [ok, just, looks, nice, modern, outside, desk,...
20489    [hotel, theft, ruined, vacation, hotel, opened...
20490    [people, talking, ca, believe, excellent, rati...
Name: Review, Length: 20491, dtype: object


In [173]:
len_val = int(len(text_token) * prop_val)
review_text =text_token[len_val:]

model = gensim.models.Word2Vec(
    review_text,
    window=10,
    min_count=2,
    workers=4,
)        # min no of times for word occurs to be included

In [174]:
model.wv.most_similar(positive='awesome', topn=10)

[('amazing', 0.8797987103462219),
 ('incredible', 0.8473653793334961),
 ('wonderful', 0.7539464831352234),
 ('gorgeous', 0.7512045502662659),
 ('beautiful', 0.7449389100074768),
 ('fantastic', 0.7134550213813782),
 ('terrific', 0.7030385732650757),
 ('loved', 0.6980385780334473),
 ('lenin', 0.6905674934387207),
 ('fabulous', 0.6849570274353027)]

In [175]:
model.train(review_text, total_examples=model.corpus_count, epochs=model.epochs)
model.wv.most_similar("terrible")


[('horrible', 0.8324288129806519),
 ('awful', 0.8249699473381042),
 ('worst', 0.7549934387207031),
 ('poor', 0.6796994209289551),
 ('worse', 0.6680483222007751),
 ('poisoning', 0.667526125907898),
 ('disgusting', 0.6544516682624817),
 ('bad', 0.6509873867034912),
 ('lousy', 0.6078166365623474),
 ('gross', 0.6055698990821838)]

In [176]:
index_to_key = model.wv.index_to_key # different in older versions

print(f"index_to_key is a list with {len(index_to_key)} words, sample is:")
for index, word in enumerate(index_to_key):
    if index == 10:
        break
    print(f"\tword {index} == {word}")

index_to_key is a list with 22477 words, sample is:
	word 0 == hotel
	word 1 == room
	word 2 == not
	word 3 == great
	word 4 == good
	word 5 == staff
	word 6 == stay
	word 7 == did
	word 8 == rooms
	word 9 == just


In [177]:
from keras.preprocessing.sequence import pad_sequences
word_index = {w: i+1 for i,w in enumerate(index_to_key) if i < max_words-1}  # Keep just max_words (zero is reserved for unknown)


sequences = [[word_index.get(w, 0) for w in sent] for sent in reviews] # code the sentences

seqs_truncated = pad_sequences(sequences, maxlen=max_review_length, padding="pre", truncating="post")

ratings = np.asarray(Ratings) 

# prepare training and validation data
x_val = seqs_truncated[:len_val]
partial_x_train = seqs_truncated[len_val:]

y_val = to_categorical(ratings[:len_val]-1, num_classes=5)
partial_y_train = to_categorical(ratings[len_val:]-1, num_classes=5)

print('Length of validation set =', len(x_val))
print('Length of training set =', len(partial_x_train))
    

Length of validation set = 4098
Length of training set = 16393


In [199]:
def sample_network(embedding):
    network = Sequential()
    network.add(embedding)
    network.add(Bidirectional(LSTM(64)))
    network.add(Dense(64, activation='relu'))
    network.add(Dense(32, activation="relu"))
    network.add(Dense(16, activation="relu"))
    network.add(Dropout(0.5))
    network.add(Dense(5,activation='softmax'))
    return network

In [200]:
# Function returns wv_matrix ready for a Keras Embeddings layer.  wv is the mapping from word to vector.
def load_embeddings_into_wv(wv):
    
    wv_matrix = np.zeros((max_words, embedding_size))
    for word, i in word_index.items():
        if i >= max_words:  
            continue
        try:
            embedding_vector = wv[word]
            wv_matrix[i] = embedding_vector
        except:
            pass    
    return wv_matrix

In [201]:
wv_matrix = load_embeddings_into_wv(model.wv)
embedding =Embedding(max_words,embedding_size, input_length=max_review_length, weights = [wv_matrix],trainable = True)
network_wordvec = sample_network(embedding)
network_wordvec.summary()
network_wordvec.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])


Model: "sequential_23"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_24 (Embedding)     (None, 100, 300)          1500000   
_________________________________________________________________
bidirectional_5 (Bidirection (None, 128)               186880    
_________________________________________________________________
dense_16 (Dense)             (None, 64)                8256      
_________________________________________________________________
dense_17 (Dense)             (None, 32)                2080      
_________________________________________________________________
dense_18 (Dense)             (None, 16)                528       
_________________________________________________________________
dropout_10 (Dropout)         (None, 16)                0         
_________________________________________________________________
dense_19 (Dense)             (None, 5)               

In [202]:
hist_word_vec =network_wordvec.fit(partial_x_train,partial_y_train, epochs=no_epochs, 
                   batch_size=256, validation_data=(x_val,y_val))

Epoch 1/8
65/65 [==============================] - 39s 606ms/step - loss: 1.5886 - accuracy: 0.4449 - val_loss: 1.5750 - val_accuracy: 0.3875
Epoch 2/8
65/65 [==============================] - 38s 587ms/step - loss: 1.5491 - accuracy: 0.4554 - val_loss: 1.5458 - val_accuracy: 0.3875
Epoch 3/8
65/65 [==============================] - 37s 564ms/step - loss: 1.5152 - accuracy: 0.4554 - val_loss: 1.5215 - val_accuracy: 0.3875
Epoch 4/8
10/65 [===>..........................] - ETA: 30s - loss: 1.4995 - accuracy: 0.4484

KeyboardInterrupt: 

In [140]:
partial_x_train

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [141]:
partial_y_train

array([[0., 0., 1., 0., 0.],
       [0., 0., 0., 1., 0.],
       [0., 0., 1., 0., 0.],
       ...,
       [0., 1., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0.]], dtype=float32)